In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import nn

data_from_xlsx=pd.read_excel(r'data.xlsx',sheet_name=(['train','test']),usecols=[0,1,2,3,4,5,6],)
data_train=data_from_xlsx['train']  ##.value numpy
data_test=data_from_xlsx['test']
data_train_target=data_train['target'].values
data_test_target=data_test['target'].values
data_train=data_train[['a','b','c','d','e','f']].values
data_test=data_test[['a','b','c','d','e','f']].values
data_train=torch.from_numpy(data_train).float()
data_test=torch.from_numpy(data_test).float()
# print(data_train.size(),data_train.dtype)
data_train_target=torch.from_numpy(data_train_target).float()
data_test_target=torch.from_numpy(data_test_target).float()


def get_data(train_data,test_data,train_data_target,test_data_target,batch_size):
    train_ds=TensorDataset(train_data,train_data_target)
    test_ds = TensorDataset(test_data, test_data_target)
    return DataLoader(train_ds,batch_size=batch_size,shuffle=True,drop_last=True),DataLoader(test_data,batch_size=batch_size)


def loss_batch(model,loss_func,x_batch,y_batch,opt=None):
    print(model(x_batch).squeeze(-1).dtype,y_batch.dtype)
    loss=loss_func(model(x_batch).squeeze(-1),y_batch.float())
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(),len(x_batch)


def fit(steps,model,loss_func,opt,train_dl,test_dl):
    for step in range(steps):
        model.train()
        for x_batch,y_batch in train_dl:
            loss_batch(model,loss_func,x_batch,y_batch,opt)

        model.eval()
        with torch.no_grad():
            losses,nums=zip(*[loss_batch(model,loss_func,x_b,y_b) for x_b,y_b in test_dl])
        test_loss=np.sum(np.multiply(losses,nums))/np.sum(nums)
        print('当前step: '+str(step),'测试集损失: '+str(test_loss))

class basic_bp(nn.Module):
    def __init__(self):
        super(basic_bp, self).__init__()
        self.fc1=nn.Linear(6,4)
        # self.relu=nn.ReLU()
        self.fc2=nn.Linear(4,1)
        # self.sigmoid=nn.Sigmoid()

    def forward(self,x):
        x=F.relu(self.fc1(x))

        x=F.sigmoid(self.fc2(x))

        return x




In [52]:
def get_model():
    model=basic_bp()
    opt=torch.optim.Adam(model.parameters(),lr=0.001)
    return model,opt

In [53]:
model,optim=get_model()

In [54]:
loss_fn=nn.BCELoss()

In [66]:
data_train_ds,data_test_ds=get_data(data_train,data_test,data_train_target,data_test_target,64)

In [84]:
epochs=500

In [94]:
for epoch in range(epochs):
    model.train()
    for x,y in data_train_ds:
        y_pred=model(x)
        loss=loss_fn(y_pred.squeeze(-1),y)
        optim.zero_grad()
        loss.backward()
        optim.step()

In [95]:
model.eval()

basic_bp(
  (fc1): Linear(in_features=6, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [96]:
with torch.no_grad():
    loss=loss_fn(model(data_train).squeeze(-1),data_train_target).item()

In [97]:
loss

0.45512741804122925

In [98]:
loss_fn(model(data_train).squeeze(-1),data_train_target)

tensor(0.4551, grad_fn=<BinaryCrossEntropyBackward0>)

In [128]:
res=model(data_train).squeeze(-1)

In [129]:
res=(res>=0.5)

In [130]:
res=(res==data_train_target)

In [131]:
hit=res.sum()

In [132]:
acc=(hit/len(data_train_target)).item()

In [133]:
acc

0.8240740895271301

In [134]:
res_test=model(data_test).squeeze(-1)

In [136]:
res_test=(res_test>=0.5)

In [137]:
res_test=(res_tess==data_test_target)

In [139]:
acc_test=(res_test.sum()/len(data_test_target)).item()

In [140]:
acc_test

0.800000011920929

In [ ]:
无归一化 6-4-1 bp神经网络  初步acc (train:0.824 test:0.800)

In [5]:
torch.zeros(4,1)

tensor([[0.],
        [0.],
        [0.],
        [0.]])

In [7]:
torch.tensor(torch.tensor(5))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_356\68745289.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(torch.tensor(5))


tensor(5)

In [8]:
aaa=[]
tunal_FIT=1


In [10]:
for i in range(10):
    aaa.append(10)
    if aaa[i]>1:
        print(13)

13
13
13
13
13
13
13
13
13
13


In [12]:
torch.tensor([1,2,3])*torch.tensor([2,2,6])

tensor([ 2,  4, 18])

In [2]:
import copy

In [1]:
a=[1,23,3]

In [3]:
b=copy.deepcopy(a)

In [4]:
id(a),id(b)

(2340608988352, 2340611471552)

In [5]:
b=a

In [6]:
id(a),id(b)

(2340608988352, 2340608988352)

In [1]:
import torch

In [3]:
def fojb(vec_tensor):
    x1=vec_tensor[0].item()
    x2=vec_tensor[1].item()
    return (x1-2)**2 + x2**2

In [15]:
fojb(torch.tensor([0.7751,2.9484]))

10.193442709226304

In [8]:
aa=torch.tensor([[1,2],[3,4]])

In [9]:
aa

tensor([[1, 2],
        [3, 4]])

In [12]:
aa[0]=torch.tensor([-3.5734,3.4192])

In [11]:
aa

tensor([[233,  33],
        [  3,   4]])

In [16]:
aaa=torch.tensor([111,3434])

In [17]:
bbb=torch.tensor([1,1])

In [18]:
aaa=bbb

In [19]:
aaa

tensor([1, 1])

In [20]:
bbb

tensor([1, 1])

In [22]:
id(aaa)==id(bbb)

True

In [23]:
import math

In [24]:
math.sin(0)

0.0

In [25]:
torch.ones(31)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [1]:
import torch

In [14]:
tt=torch.tensor([[1,2,3],[3,4,5],[6,7,8]])

In [15]:
tt=tt.float()

In [16]:
tt.shape

torch.Size([3, 3])

In [17]:
bn=torch.nn.BatchNorm1d(3)

In [19]:
zz=bn(tt)

In [20]:
zz

tensor([[-1.1355, -1.1355, -1.1355],
        [-0.1622, -0.1622, -0.1622],
        [ 1.2978,  1.2978,  1.2978]], grad_fn=<NativeBatchNormBackward0>)

In [25]:
torch.mean(zz,dim=1)

tensor([-1.1355, -0.1622,  1.2978], grad_fn=<MeanBackward1>)